In [50]:
%pip install craft-text-detector

  Using cached craft_text_detector-0.4.3-py3-none-any.whl.metadata (4.8 kB)
  Using cached opencv-python-4.5.4.60.tar.gz (89.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\1288843.TRANSIT.000\\AppData\\Local\\Temp\\pip-install-sl5bfq_a\\opencv-python_ba64db24a6034801a4e7c0457a346b4a\\opencv/samples/winrt_universal/VideoCaptureXAML/video_capture_xaml/video_capture_xaml.WindowsPhone/video_capture_xaml.WindowsPhone.vcxproj.filters'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [49]:
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
from craft_text_detector import Craft
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import torch

ModuleNotFoundError: No module named 'craft_text_detector'

In [34]:
# Class to Load images & transcriptions
class TextDataset(Dataset):
    def __init__(self, images_dir, transcriptions_dir, transform=None):
        self.images_dir = images_dir
        self.transcriptions_dir = transcriptions_dir
        self.transform = transform
        self.image_filenames = os.listdir(images_dir)

    def parse_transcrpition(self, transcription_file):
        bounding_boxes = []
        texts = []

        with open(transcription_file, 'r') as file:
            for line in file:
                parts = line.strip().split(',')
                coords = list(map(int, parts[:8]))
                text = parts[8]
                bounding_boxes.append(coords)
                texts.append(text)
            
        return bounding_boxes, texts
    
    def __len__(self):
        return len(self.image_filenames)
    
    def __getitem__(self, index):
        image_name = self.image_filenames[index]
        image_path = os.path.join(self.images_dir, image_name)
        transcription_path = os.path.join(self.transcriptions_dir, image_name.replace(".jpg", '.txt'))
        image = Image.open(image_path).convert("RGB")
        bounding_boxes, texts = self.parse_transcrpition(transcription_path)

        if self.transform:
            image = self.transform(image)

        return image, bounding_boxes, texts
        

In [35]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = TextDataset(images_dir="./dataset/test/test_images/", transcriptions_dir="./dataset/test/test_localization_transcription/")
data_loader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)

In [36]:
def visualize_bounding_boxes(image_path, bounding_boxes, texts):
    image = cv2.imread(image_path)

    for box, text in zip(bounding_boxes, texts):
        points = np.array([[box[i], box[i + 1]] for i in range(0, 8, 2)], np.int32)
        points = points.reshape((-1, 1, 2))
        cv2.polylines(image, [points], isClosed=True, color=(0, 255, 0), thickness=2)
        cv2.putText(image, text, (box[0], box[1] - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)

    cv2.imshow("Image with bounding Boxes", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [44]:
image_path = "./dataset/test/test_images/img_100.jpg"
bounding_boxes, texts = dataset.parse_transcrpition('./dataset/test/test_localization_transcription/gt_img_100.txt')
visualize_bounding_boxes( image_path, bounding_boxes, texts)